# **Feature Engineering**:<a class="anchor" id="0"></a>

1. [**Data Import**](#1)
2. [**Feature Engineering**](#2)
3. [**Feature Pickling**](#3)

In [1]:
import numpy as np
import pandas as pd

from _util.custom_plotting import *
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from _util.custom_mem_opt import custom_mem_opt

import category_encoders as ce
import gc

sns.set_style('darkgrid')
color = sns.color_palette()

### Data Import <a class=anchor id="1"></a>
[Back to top](#0)

Apply memory optimization.

In [3]:
root = './_data/'

aisles = custom_mem_opt(pd.read_csv(root + 'aisles.csv'), verbose=False)
departments = custom_mem_opt(pd.read_csv(root + 'departments.csv'), verbose=False)
orders = custom_mem_opt(pd.read_csv(root + 'orders.csv'), verbose=False)
order_products_prior = custom_mem_opt(pd.read_csv(root + 'order_products__prior.csv'), verbose=False)
order_products_train = custom_mem_opt(pd.read_csv(root + 'order_products__train.csv'), verbose=False)
products = custom_mem_opt(pd.read_csv(root + 'products.csv'), verbose=False)

In [4]:
prior_df = order_products_prior.merge(orders, on ='order_id', how='inner')
prior_df = prior_df.merge(products, on = 'product_id', how = 'left')
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


### Feature Engineering <a class=anchor id="2"></a>
[Back to top](#0)

Start with purchase frequency.

In [6]:
prior_df['user_buy_product_times'] = prior_df.groupby(['user_id', 'product_id']).cumcount() + 1
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


### Customer Level metrics
- Customer's average and std day-of-week of order
- Customer's average and std hour-of-day of order
- Customer's average and std days-since-prior-order
- Total orders by a customer
- Total products customer has bought
- Total unique products customer has bought
- Customer's total re-ordered products
- Customer's overall re-order percentage
- Average order size of a customer
- Customer's mean of reordered items of all orders
- Percentage of reordered itmes in customer's last three orders
- Total orders in customer's last three orders

In [8]:
print(prior_df.isnull().any())

order_id                  False
product_id                False
add_to_cart_order         False
reordered                 False
user_id                   False
eval_set                  False
order_number              False
order_dow                 False
order_hour_of_day         False
days_since_prior_order     True
product_name              False
aisle_id                  False
department_id             False
user_buy_product_times    False
dtype: bool


In [9]:
# when no prior order, the value is null. Imputing as 0
prior_df.days_since_prior_order = prior_df.days_since_prior_order.fillna(0)

In [10]:
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [11]:
cust_feats = prior_df.groupby('user_id').agg(
    avg_dow=('order_dow', 'mean'),
    std_dow=('order_dow', 'std'),
    avg_doh=('order_hour_of_day', 'mean'),
    std_doh=('order_hour_of_day', 'std'),
    avg_since_order=('days_since_prior_order', 'mean'),
    std_since_order=('days_since_prior_order', 'std'),
    total_orders_by_user=('order_number', lambda x: x.nunique()),
    total_products_by_user=('product_id', 'count'),
    total_unique_product_by_user=('product_id', lambda x: x.nunique()),
    total_reorders_by_user=('reordered', 'sum'),
    reorder_propotion_by_user=('reordered', 'mean')
)

In [12]:
cust_feats.reset_index(inplace = True)
cust_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559065,10,59,18,41,0.694915
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378


In [13]:
cust_feats_2 = prior_df.groupby(['user_id', 'order_number']).agg(
    average_order_size=('reordered', 'count'),
    reorder_in_order=('reordered', 'mean')
)
cust_feats_2.reset_index(inplace = True)
cust_feats_2.head()

,user_id,order_number,average_order_size,reorder_in_order
0,1,1,5,0.000
1,1,2,6,0.500
2,1,3,5,0.600
3,1,4,5,1.000
4,1,5,8,0.625


In [14]:
cust_feats_3 = cust_feats_2.groupby('user_id').agg({'average_order_size' : 'mean', 
                                   'reorder_in_order':'mean'})
cust_feats_3 = cust_feats_3.reset_index()
cust_feats_3.head()

,user_id,average_order_size,reorder_in_order
0,1,5.900000,0.705833
1,2,13.928571,0.447961
2,3,7.333333,0.658817
3,4,3.600000,0.028571
4,5,9.250000,0.377778


In [15]:
cust_feats = cust_feats.merge(cust_feats_3, on = 'user_id', how = 'left')
cust_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559065,10,59,18,41,0.694915,5.900000,0.705833
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923,13.928571,0.447961
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000,7.333333,0.658817
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556,3.600000,0.028571
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378,9.250000,0.377778


In [16]:
last_three_orders = cust_feats_2.groupby('user_id')['order_number'].nlargest(3).reset_index()
last_three_orders.head()

,user_id,level_1,order_number
0,1,9,10
1,1,8,9
2,1,7,8
3,2,23,14
4,2,22,13


In [17]:
last_three_orders = cust_feats_2.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
last_three_orders.head()

,user_id,order_number,average_order_size,reorder_in_order,level_1
0,1,8,6,0.666667,7
1,1,9,6,1.000000,8
2,1,10,9,0.666667,9
3,2,12,19,0.578947,21
4,2,13,9,0.000000,22


In [18]:
last_three_orders['rank'] = last_three_orders.groupby("user_id")["order_number"].rank("dense", ascending=True)

In [19]:
last_order_feats = last_three_orders.pivot_table(index = 'user_id', columns = ['rank'], \
                                                 values=['average_order_size', 'reorder_in_order']).\
                                                reset_index(drop = False)
last_order_feats.columns = ['user_id','orders_3', 'orders_2', 'orders_1', 'reorder_3', 'reorder_2', 'reorder_1']
last_order_feats.head()

,user_id,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,6.0,6.0,9.0,0.666667,1.0,0.666667
1,2,19.0,9.0,16.0,0.578947,0.0,0.625000
2,3,6.0,5.0,6.0,0.833333,1.0,1.000000
3,4,7.0,2.0,3.0,0.142857,0.0,0.000000
4,5,9.0,5.0,12.0,0.444444,0.4,0.666667


In [20]:
cust_feats = cust_feats.merge(last_order_feats, on = 'user_id', how = 'left')
cust_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559065,10,59,18,41,0.694915,5.900000,0.705833,6.0,6.0,9.0,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923,13.928571,0.447961,19.0,9.0,16.0,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000,7.333333,0.658817,6.0,5.0,6.0,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556,3.600000,0.028571,7.0,2.0,3.0,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378,9.250000,0.377778,9.0,5.0,12.0,0.444444,0.4,0.666667


### Customer x Product Level
- Customer's avg add-to-cart-order for a product
- Customer's avg days_since_prior_order for a product
- Customer's product total orders, reorders and reorders percentage
- Customer's order number when the product was bought last
- Customer's product purchase history of last three orders

In [22]:
cust_product_feats = prior_df.groupby(['user_id', 'product_id']).agg(
    total_product_orders_by_user=('reordered', 'count'),
    total_product_reorders_by_user=('reordered', 'sum'),
    user_product_reorder_percentage=('reordered', 'mean'),
    avg_add_to_cart_by_user=('add_to_cart_order', 'mean'),
    avg_days_since_last_bought=('days_since_prior_order', 'mean'),
    last_ordered_in=('order_number', 'max')
)

In [23]:
cust_product_feats.reset_index(inplace = True)
cust_product_feats.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in
0,1,196,10,9,0.900000,1.400000,17.600000,10
1,1,10258,9,8,0.888889,3.333333,19.555555,10
2,1,10326,1,0,0.000000,5.000000,28.000000,5
3,1,12427,10,9,0.900000,3.300000,17.600000,10
4,1,13032,3,2,0.666667,6.333333,21.666666,10


In [24]:
last_three_orders.head()

,user_id,order_number,average_order_size,reorder_in_order,level_1,rank
0,1,8,6,0.666667,7,1.0
1,1,9,6,1.000000,8,2.0
2,1,10,9,0.666667,9,3.0
3,2,12,19,0.578947,21,1.0
4,2,13,9,0.000000,22,2.0


In [25]:
last_orders = prior_df.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
last_orders.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times,average_order_size,reorder_in_order,level_1,rank
0,7,34050,1,0,142903,prior,11,2,14,30.0,Orange Juice,31,7,1,2,0.000000,2231251,2.0
1,7,46802,2,0,142903,prior,11,2,14,30.0,Pineapple Chunks,116,1,1,2,0.000000,2231251,2.0
2,14,20392,1,1,18194,prior,49,3,15,3.0,Hair Bender Whole Bean Coffee,26,7,1,11,0.818182,282882,1.0
3,14,27845,2,1,18194,prior,49,3,15,3.0,Organic Whole Milk,84,16,1,11,0.818182,282882,1.0
4,14,162,3,1,18194,prior,49,3,15,3.0,Organic Mini Homestyle Waffles,52,1,1,11,0.818182,282882,1.0


In [26]:
last_orders['rank'] = last_orders.groupby(['user_id', 'product_id'])['order_number'].rank("dense", ascending=True)

In [27]:
product_purchase_history = last_orders.pivot_table(index = ['user_id', 'product_id'],\
                                                   columns='rank', values = 'reordered').reset_index()
product_purchase_history.columns = ['user_id', 'product_id', 'is_reorder_3', 'is_reorder_2', 'is_reorder_1']
product_purchase_history.fillna(0, inplace = True)
product_purchase_history.head()

,user_id,product_id,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,1.0,1.0,1.0
1,1,10258,1.0,1.0,1.0
2,1,12427,1.0,1.0,1.0
3,1,13032,1.0,0.0,0.0
4,1,25133,1.0,1.0,1.0


In [28]:
cust_product_feats = cust_product_feats.merge(product_purchase_history, on=['user_id', 'product_id'], how = 'left')
cust_product_feats.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,17.600000,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.555555,10,1.0,1.0,1.0
2,1,10326,1,0,0.000000,5.000000,28.000000,5,NaN,NaN,NaN
3,1,12427,10,9,0.900000,3.300000,17.600000,10,1.0,1.0,1.0
4,1,13032,3,2,0.666667,6.333333,21.666666,10,1.0,0.0,0.0


In [29]:
print(cust_product_feats.isnull().sum())

user_id                                  0
product_id                               0
total_product_orders_by_user             0
total_product_reorders_by_user           0
user_product_reorder_percentage          0
avg_add_to_cart_by_user                  0
avg_days_since_last_bought               0
last_ordered_in                          0
is_reorder_3                       8382738
is_reorder_2                       8382738
is_reorder_1                       8382738
dtype: int64


In [30]:
cust_product_feats.fillna(0, inplace = True)